##### Data Preprocessing

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import nltk
import tensorflow as tf
import os
import string
import re
data = pd.read_excel('sentence_past_present_future.xlsx')
y = data['Categorie']
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.text import one_hot
voc_size = 10000
input = []
for x in data['Sentences']:
    val = word_tokenize(x)
    one_rpr = one_hot(str(val),n=voc_size,lower=False,split=None)
    input.append(one_rpr)
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
sent_length = 15
embedded_docs = pad_sequences(input,padding='post',maxlen=sent_length)
dim = 10
model =Sequential()
model.add(Embedding(voc_size,dim,input_length=sent_length))
model.compile('adam','mse')
model.summary()
x = model.predict(embedded_docs)
rows = x.shape[0] 
cols = x.shape[1] * x.shape[2]
x = x.reshape(rows,cols)
from sklearn.model_selection import train_test_split
x_train , x_test , y_train , y_test = train_test_split(x,y,test_size=0.25,random_state=42)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 15, 10)            100000    
                                                                 
Total params: 100000 (390.62 KB)
Trainable params: 100000 (390.62 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
87/87 [==============================] - 0s 1ms/step


#### Model

In [2]:
import xgboost as xgb

In [3]:
model = xgb.XGBClassifier(objective='multi:softmax',  
                          eval_metric='logloss',     
                          use_label_encoder=False,
                         num_class=2)

In [4]:
model.fit(x_train,y_train)

C:\Users\Divyank\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\training.py:183: UserWarning: [23:51:25] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None, num_class=2, ...)

In [5]:
from sklearn.metrics import accuracy_score , confusion_matrix , precision_score , recall_score , f1_score , roc_auc_score
def Evaluation_metrics(actual,pred):
    accuracy_sco = accuracy_score(actual,pred)
    confusion_mat = confusion_matrix(actual,pred)
    precision_sco = precision_score(actual,pred,average='macro')
    recall_sco = recall_score(actual,pred,average='macro')
    f1_sco = f1_score(actual,pred,average='macro')
    return accuracy_sco,confusion_mat,precision_sco,recall_sco,f1_sco

In [6]:
predicted_val = model.predict(x_test)

In [8]:
Evaluation_metrics(y_test,predicted_val)

(0.8701298701298701,
 array([[213,  12,   7],
        [  4, 189,  38],
        [  4,  25, 201]], dtype=int64),
 0.8723857538499303,
 0.8700661033119804,
 0.8706895484438939)

In [7]:
model.score(x_test,y_test)

0.8701298701298701

In [9]:
model.score(x_train,y_train)

1.0